In [15]:
import pandas as pd
import os

import torch
from experiments.transformers.train_pred import train , validation , test , build_model , get_data , load_model_pred , predict_price
from data.transformers.utils import DotDict

### Select Model

In [16]:
args = DotDict

args.model = 'pyraformer' # model of experiment list ['informer' , 'transformer' , 'pyraformer' ,'etsformer']

### Data Parameters

In [17]:
# Data Parameters
args.data = 'BTC' # data ['BTC' or 'ETH']
args.root_path_train = '../data/processed/train'
args.data_path_train = 'processed-data-train-BTC.csv'
args.root_path_test = '../data/processed/test'
args.data_path_test = 'processed-data-test-BTC.csv'
args.features = 'S' #  S:uni-variate predict uni-variate, MS: multivariate predict uni-variate
args.target = 'midpoint' # target feature
args.freq = 'Sec' # freq for time features encoding, options:[Sec:secondly, Min:minutely]
args.normalise = True
args.checkpoints = '../experiments/checkpoints' # location of model checkpoints
args.timestamp = True

### MLFlow

In [18]:
#MLFlow Parameters
args.experiment_name = "Pyraformer-Final"
args.run_name = "1-day-60s"

### Model Parameters

In [19]:
# Transformer Input Parameters
args.seq_len = 180 # input sequence length of Informer encoder
args.label_len = 60 # start token length of Informer decoder
args.pred_len = 60 # prediction sequence length
# Informer decoder input: concat[start token series(label_len), zero padding series(pred_len)]

# Transformer Model Parameters
args.enc_in = 1 # encoder input size
args.dec_in = 1 # decoder input size
args.c_out = 1 # output size

args.d_model = 512 # dimension of model
args.d_ff = 1024 # dimension of fcn in model
args.n_heads = 8 # num of heads
args.e_layers = 2 # num of encoder layers
args.d_layers = 2 # num of decoder layers

args.dropout = 0.05 # dropout
args.attn = 'prob' # attention used in encoder, options:[prob, full]
args.embed = 'timeF' # time features encoding, options:[timeF, fixed, learned]
args.activation = 'gelu' # activation
args.distil = True # whether to use distilling in encoder
args.output_attention = False # whether to output attention in ecoder
args.mix = True
args.padding = 0

#informer
args.factor = 5 # probsparse attn factor


# Pyraformer args
args.window_size = [4,4]
args.inner_size = 3


### Training Parameters

In [20]:
args.batch_size = 30
args.learning_rate = 0.0001
args.loss = 'mse'
args.lr_adj = 'type1'
args.use_amp = False # automatic mixed precision training

args.num_workers = 0
args.itr = 1
args.train_epochs = 20
args.patience = 3
args.des = 'exp'

args.use_gpu = True if torch.cuda.is_available() else False
args.gpu = 0

if args.use_gpu and torch.cuda.is_available():
    args.device = torch.device(f'cuda:{args.gpu}')
else:
    args.device = torch.device('cpu')

In [21]:
print(args.device)

cuda:0


In [22]:
filtered_args = {k: v for k, v in args.__dict__.items() if not k.startswith('__') and not k.endswith('__')}

print('Args in experiment:')
print(filtered_args)

Args in experiment:
{'model': 'pyraformer', 'data': 'BTC', 'root_path_train': '../data/processed/train', 'data_path_train': 'processed-data-train-BTC.csv', 'root_path_test': '../data/processed/test', 'data_path_test': 'processed-data-test-BTC.csv', 'features': 'S', 'target': 'midpoint', 'freq': 'Sec', 'normalise': True, 'checkpoints': '../experiments/checkpoints', 'timestamp': True, 'experiment_name': 'Pyraformer-Final', 'run_name': '1-day-60s', 'seq_len': 180, 'label_len': 60, 'pred_len': 60, 'enc_in': 1, 'dec_in': 1, 'c_out': 1, 'd_model': 512, 'd_ff': 1024, 'n_heads': 8, 'e_layers': 2, 'd_layers': 2, 'dropout': 0.05, 'attn': 'prob', 'embed': 'timeF', 'activation': 'gelu', 'distil': True, 'output_attention': False, 'mix': True, 'padding': 0, 'factor': 5, 'window_size': [4, 4], 'inner_size': 3, 'batch_size': 30, 'learning_rate': 0.0001, 'loss': 'mse', 'lr_adj': 'type1', 'use_amp': False, 'num_workers': 0, 'itr': 1, 'train_epochs': 20, 'patience': 3, 'des': 'exp', 'use_gpu': True, 'gpu

In [23]:
ii = 1
setting = '{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_at{}_fc{}_eb{}_dt{}_mx{}_{}_{}'.format(args.model, args.data, args.features,
            args.seq_len, args.label_len, args.pred_len,
            args.d_model, args.n_heads, args.e_layers, args.d_layers, args.d_ff, args.attn, args.factor, args.embed, args.distil, args.mix, args.des, ii)

print(setting)

pyraformer_BTC_ftS_sl180_ll60_pl60_dm512_nh8_el2_dl2_df1024_atprob_fc5_ebtimeF_dtTrue_mxTrue_exp_1


In [24]:
model = build_model(args)
model.to(args.device)


# Calculate number of parameters
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Number of parameters: {num_params}")

Number of parameters: 4601276


In [25]:

# Train the model
print('>>>>>>>start training: {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(args.run_name))
model, args = train(model=model, args=args, setting=setting)

# Test the model
print('>>>>>>>testing: {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(args.run_name))
test(model=model, args=args, setting=setting)

# Clear the GPU cache
torch.cuda.empty_cache()

>>>>>>>start training: 1-day-60s>>>>>>>>>>>>>>>>>>>>>>>>>>
train 54389
val 13598


100%|██████████| 1812/1812 [01:57<00:00, 15.42it/s]


Epoch: 1 cost time: 117.49671578407288
Epoch: 1, Steps: 1812 | Train Loss: 0.0171172 Vali Loss: 0.0077122
Validation loss decreased (inf --> 0.007712). Saving model...
Updating learning rate to 0.0001


100%|██████████| 1812/1812 [01:56<00:00, 15.51it/s]


Epoch: 2 cost time: 116.839022397995
Epoch: 2, Steps: 1812 | Train Loss: 0.0123992 Vali Loss: 0.0054162
Validation loss decreased (0.007712 --> 0.005416). Saving model...
Updating learning rate to 5e-05


...

Epoch: 15 cost time: 113.22070932388306
Epoch: 15, Steps: 1812 | Train Loss: 0.0090639 Vali Loss: 0.0041689
EarlyStopping counter: 3 out of 3
Early stopping


C:\Users\misko\OneDrive - University of Leeds\Year 4\Final Year Project\GitHub\src\data\transformers\data_loader.py:181: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stamp['date'] = pd.to_datetime(df_stamp.date)


>>>>>>>testing: 1-day-60s<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
test 16833
loading model


100%|██████████| 561/561 [00:18<00:00, 29.75it/s]


test shape: (561, 30, 60, 1) (561, 30, 60, 1)
test shape: (16830, 60, 1) (16830, 60, 1)
MAE: 0.1478, MSE: 0.0860, RMSE: 0.2932, MAPE: 56.0683%, MSPE: 2141.6636%


In [10]:
# Define the settings
pred_lens = [30,60, 180]

# Loop over the prediction lengths
for pred_len in pred_lens:
    args.seq_len = 300 # input sequence length of Informer encoder
    args.label_len = 90 # start token length of Informer decoder

    # Set the arguments for this run
    run_name = "300-lookback-1-day-{}s".format(pred_len)
    args.run_name = run_name
    args.pred_len = pred_len

    setting = '{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_at{}_fc{}_eb{}_dt{}_mx{}_{}_{}'.format(args.model, args.data, args.features,
            args.seq_len, args.label_len, args.pred_len,
            args.d_model, args.n_heads, args.e_layers, args.d_layers, args.d_ff, args.attn, args.factor, args.embed, args.distil, args.mix, args.des, ii)


    model = build_model(args)
    model.to(args.device)


    # Train the model
    print('>>>>>>>start training: {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(run_name))
    model, args = train(model=model, args=args, setting=setting)

    # Test the model
    print('>>>>>>>testing: {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(run_name))
    test(model=model, args=args, setting=setting)

    # Clear the GPU cache
    torch.cuda.empty_cache()

>>>>>>>start training: 300-lookback-1-day-30s>>>>>>>>>>>>>>>>>>>>>>>>>>
train 54299
val 13628


100%|██████████| 1809/1809 [03:27<00:00,  8.70it/s]


Epoch: 1 cost time: 207.84145402908325
Epoch: 1, Steps: 1809 | Train Loss: 0.0106400 Vali Loss: 0.0027642
Validation loss decreased (inf --> 0.002764). Saving model...
Updating learning rate to 0.0001


100%|██████████| 1809/1809 [03:26<00:00,  8.76it/s]


Epoch: 2 cost time: 206.40900921821594
Epoch: 2, Steps: 1809 | Train Loss: 0.0064703 Vali Loss: 0.0024956
Validation loss decreased (0.002764 --> 0.002496). Saving model...
Updating learning rate to 5e-05


...

Epoch: 8 cost time: 204.50883674621582
Epoch: 8, Steps: 1804 | Train Loss: 0.0281211 Vali Loss: 0.0157712
EarlyStopping counter: 3 out of 3
Early stopping


C:\Users\misko\OneDrive - University of Leeds\Year 4\Final Year Project\GitHub\src\data\transformers\data_loader.py:181: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stamp['date'] = pd.to_datetime(df_stamp.date)


>>>>>>>testing: 300-lookback-1-day-180s<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
test 16593
loading model


100%|██████████| 553/553 [00:32<00:00, 17.00it/s]


test shape: (553, 30, 180, 1) (553, 30, 180, 1)
test shape: (16590, 180, 1) (16590, 180, 1)
MAE: 0.3045, MSE: 0.2304, RMSE: 0.4800, MAPE: 116.0510%, MSPE: 6029.7405%
